In [21]:
from glob import glob
import numpy as np

n_rounds = 6
prec = 1e-2

class atom2():
    def __init__(self, label, spatial_coord, group = None, charge=None):
        self.sc = spatial_coord # numpy 
        self.label = label
        self.g = group
        self.q = charge
    def __eq__(self, other):
        if isinstance(other, atom2):
            return self.label.lower() == other.label.lower() and sum(self.sc == other.sc) == 3
        return NotImplemented
    def __sub__(self, other):
        if isinstance(other, np.ndarray):
            return atom2(self.label, self.sc - other, self.g, self.q)
    def __add__(self, other):
        if isinstance(other, np.ndarray):
            return atom2(self.label, self.sc + other, self.g, self.q)
    def __mul__(self, other):
        if isinstance(other, np.ndarray):
            return atom2(self.label, np.dot(other, self.sc), self.g, self.q) if np.shape(other)[0] == 3 and np.shape(other)[1] else self
        if isinstance(other, (int, float)):
            return atom2(self.label, other * self.sc, self.g, self.q)

def atom2string(atom_obj):
    sp_coord = ""
    for coord in atom_obj.sc:
        sp_coord += str(coord) + " "
    return atom_obj.label + " " + (str(atom_obj.q) + " " if atom_obj.label.lower() == "q" else "") + sp_coord

def eq_with_precision(atom_obj_1, atom_obj_2, precision):
    return True if sum(np.abs(atom_obj_1.sc - atom_obj_2.sc) < precision) == 3 and atom_obj_1.label.lower() == atom_obj_2.label.lower() else False

def writer2file(atom_set, name, group=False):
    with open(name + ".xyz", "w") as outtxt:
        for atom_obj in atom_set:
            outtxt.write(atom2string(atom_obj) + (atom_obj.g if group and atom_obj.g != None else "") + "\n")

def findID(atomsets, des_atom, precision=0):
    indexes = []
    for i in range(len(atomsets)):
        if ((atomsets[i] == des_atom) if precision == 0 else (eq_with_precision(atomsets[i], des_atom, precision))):
            indexes.append(i)
        else:
            continue
    return indexes

def removeREP(atomsets):
    atomsets_new = atomsets
    for atom in atomsets:
        index_list = findID(atomsets_new, atom)[::-1]
        n = len(index_list)
        for i in index_list[:-1]:
            atomsets_new.pop(i)
    return atomsets_new

def reader_out(filename):
    lat_flag, lat_i = "DIRECT LATTICE VECTORS CARTESIAN COMPONENTS (ANGSTROM)", []
    prcell_flag, prcell_i = "CARTESIAN COORDINATES - PRIMITIVE CELL", []
    lat_set = []
    prcell_set = []
    with open(filename, "r") as inputtxt:
        for i, row in enumerate(inputtxt):
            #
            if lat_flag in row:
                lat_i, lat_set = [i + 2, i + 4], []
            if len(lat_i) > 0: 
                if min(lat_i) <= i <= max(lat_i):
                    row_tmp = row.strip().split()
                    lat_set.append(np.array(row_tmp[:3], dtype=float))
            #
            if prcell_flag in row: # Put PrCell information here!
                prcell_i, prcell_set = [i + 4, i + 8], []
            if len(prcell_i) > 0: 
                if min(prcell_i) <= i <= max(prcell_i):
                    row_tmp = row.strip().split()
                    prcell_set.append(atom2(row_tmp[2], np.array(row_tmp[3:6], dtype = float)))
            #
    return lat_set, prcell_set

def builder_supercell(l_option_a, l_option_b, l_option_c, lat_set, prcell_set):
    atoms_set = []
    with open("super_cell.xyz", "w") as tmptxt:
        for n_a in l_option_a:
            for n_b in l_option_b:
                for n_c in l_option_c:
                    for atom_obj in prcell_set:
                        atoms_set.append(atom_obj + (lat_set[0]*n_a + lat_set[1]*n_b + lat_set[2]*n_c))
                        tmptxt.write(atom2string(atoms_set[-1])  + "\n")
    return atoms_set

def cutter_cluster(atoms_set, n_atominprcell, n_waves):
    waves_couner, disp = 1, 0.1
    atom_init = atoms_set[n_atominprcell]
    atom_init.g = 0
    TMP_atomwave, TMP_newwave, atoms_set_num = [atom_init], [], [atom_init]
    l_range = [1.99800, 2.82560] #[2.48754, 3.45796, 3.64001] # Must be changed for new.out!
    norm = lambda np_vector : (sum(np_vector**2))**0.5
    while waves_couner < n_waves:
        for inwaveatom in TMP_atomwave:
            for i in findID(atoms_set, inwaveatom):
                atoms_set.pop(i)
            for i in range(len(atoms_set)):
                if min(l_range) - disp <= norm(atoms_set[i].sc - inwaveatom.sc) <= max(l_range) + disp:
                    TMP_newwave.append(atoms_set[i])
                else:
                    continue
        TMP_atomwave = removeREP(TMP_newwave)
        waves_couner += 1
        atoms_set_num += TMP_atomwave
        TMP_newwave = []
    return removeREP(atoms_set_num)


Cluster generator options

In [22]:
Filename = glob("*.out")[-1]

latset, prcellset = reader_out(Filename)            

l_option_a = [0, -1, +1, -2, 2, -3, 3]
l_option_b = [0, -1, +1, -2, 2, -3, 3]
l_option_c = [0, -1, +1, -2, 2, -3, 3]

atomset = builder_supercell(l_option_a, l_option_b, l_option_c, latset, prcellset)
natominprcell = findID(atomset, prcellset[0])[0]

nwaves = 5
atomwaves = cutter_cluster(atomset, natominprcell, nwaves) # The last argument must be chosen in accordance with the supercell.

with open("supcell_sup_waves" + str(nwaves) + ".xyz", "w") as xyztxt:
    for atom_obj in atomwaves:
        xyztxt.write(atom2string(atom_obj) + "\n")

.xyz subtraction.
To read .xyz files:

In [24]:
from glob import glob
pathway = "C:\\WD_PNPI\\Cluster_workspace\\"

xyz_files = glob(pathway + "*.xyz")

for i in range(len(xyz_files)):
    print(str(i) + " : " + xyz_files[i].split("\\")[-1].split(".")[0])

0 : 1_BaMC
1 : 2_BaPse
2 : 3_BaPCh
3 : supcell_sup_waves5
4 : super_cell


Main functions:

In [12]:
def reader_xyz2(filename):
    atoms_set = []
    with open(filename, "r") as xyztxt:
        for row in xyztxt:
            row_tmp = row.strip().split()
            if len(row_tmp) > 3:
                if row_tmp[0].lower() == "q":
                    atoms_set.append(atom2(row_tmp[0], np.array(row_tmp[2:5], dtype = float), charge=float(row_tmp[1])))
                else:
                    atoms_set.append(atom2(row_tmp[0], np.array(row_tmp[1:4], dtype = float)))
            else:
                continue
    return atoms_set

def subtrack_xyz(file1, file2): # file1 - reduced >= file2 - subtracted
    atoms_set_1, atoms_set_2 = reader_xyz2(file1), reader_xyz2(file2)
    for atom_obj in atoms_set_2:
        index_list = findID(atoms_set_1, atom_obj, prec)
        for i in index_list:
            atoms_set_1.pop(i)
    writer2file(atoms_set_1, pathway + file1.split("\\")[-1].split(".")[0] + "-" + file2.split("\\")[-1].split(".")[0] + ".xyz")
    return atoms_set_1


In [ ]:
#subtrack_xyz(xyz_files[7], xyz_files[1])

System rotation

In [27]:
import numpy as np

def rotmat(direction, angle):
    from math import cos, sin, pi
    x, y, z = direction[0], direction[1], direction[2] 
    cost, sint = cos(angle/180 * pi), sin(angle/180 * pi) 
    return np.array([
        [cost + (1 - cost)*x*x, (1 - cost)*x*y - sint*z, (1 - cost)*x*z + sint*y],
        [(1 - cost)*y*x + sint*z, cost + (1 - cost)*y*y, (1 - cost)*y*z - sint*x],
        [(1 - cost)*z*x - sint*y, (1 - cost)*z*y + sint*x, cost + (1 - cost)*z*z]
    ])

def findidintrack(track, id):
    for trace in track:
        if id in trace: return True
        else: continue
    return False

def dividerpro_track(atoms_set, track, ActionMatrix, actionorder):
    track_new = []
    for trace in track:
        for atom_id in trace:
            if not findidintrack(track_new, atom_id):
                atom_obj = atoms_set[atom_id]
                trace_new = [atom_id]
                for round in range(actionorder):
                    atom_obj = atom_obj * ActionMatrix
                    list_of_ids = findID([atoms_set[i] for i in trace], atom_obj, prec)
                    if len(list_of_ids) > 0:
                        if trace_new[0] != list_of_ids[0]: trace_new.append(list_of_ids[0])
                        else:
                            track_new += [trace_new]
                            break
                    else: continue
            else: continue
    return track_new       

def findtraceintrace(trace_big, trace_small):
    containslist = [False] * len(trace_small)
    for id_b in trace_big:
        for i, id_s in enumerate(trace_small):
            if id_b == id_s: containslist[i] = True
            else: continue
    return True if sum(containslist) == len(trace_small) else False

def findtraceintrack(track_new, trace):
    for trace_new in track_new:
        if findtraceintrace(trace_new, trace): return True
        else: continue
    return False

def mergerpro_track(atoms_set, track, ActionMatrix, actionorder):
    track_new = []
    for trace in track:
        if not findtraceintrack(track_new, trace):
            trace_new = trace
            trace_tmp = trace
            for round in range(actionorder):
                trace_tmp = [findID(atoms_set, atoms_set[atom_id] * ActionMatrix, prec) for atom_id in trace_tmp]
                if not ([] in trace_tmp): trace_tmp = [ids[0] for ids in trace_tmp]
                else:
                    track_new += [trace_new]
                    break
                if not findtraceintrace(trace_new, trace_tmp): trace_new += trace_tmp
                else:
                    track_new += [trace_new]
                    break
        else: continue
    return track_new

filename = xyz_files[2]
atomsset = reader_xyz2(filename)
track0 = [[i for i in range(len(atomsset))]]
track1 = dividerpro_track(atomsset, track0, rotmat([0, 0, 1], 360/4), 4)
track2 = mergerpro_track(atomsset, track1, np.array([[1, 0, 0], [0, 1, 0], [0, 0, -1]]), 2)

def assigngroup(atomsset, track):
    for id_group, trace in enumerate(track):
        for id_atom in trace:
            atomsset[id_atom].g = ("@g" if len(trace) > 1 else "@s") + str(id_group + 1)
    return atomsset

writer2file(assigngroup(atomsset, track2), filename.split(".")[0] + "_test_g2", True)  
